# Hatef Nabili  Homework 06 Plus Proplem 1

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score

## a)

In [2]:
##Downloading punctuations and stop words for nltk tokenizer
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hatef\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hatef\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
## Loading the dataset into dataframe
yelp_df=pd.read_csv("train.csv")
yelp_df

,Rating,Review
0,1,"First of all i'm not a big fan of buffet, i tr..."
1,2,Thanks Yelp. I was looking for the words to de...
2,3,Service was so-so. They were receiving a deliv...
3,3,Stamoolis Brothers is one of the Strip Distric...
4,1,I want to give a 2 stars because the service s...
...,...,...
9995,1,Four Hours of my life I'd like to have back!\n...
9996,2,Small stadium and would have great views all a...
9997,3,I think Porktropolis has the BBQ thing nailed....
9998,5,Best CU ever!!! Great customer service!!! Know...


In [4]:
##Converts ratings to 0(negative) or 1(positive) 
def convert_label(rating):
    if rating < 4:
        return 0.0
    else:
        return 1.0

In [5]:
##Converting ratings to int
yelp_df.astype({'Rating': 'int32'}).dtypes
yelp_df['pos/neg']=yelp_df["Rating"].apply(convert_label)
yelp_df

,Rating,Review,pos/neg
0,1,"First of all i'm not a big fan of buffet, i tr...",0.0
1,2,Thanks Yelp. I was looking for the words to de...,0.0
2,3,Service was so-so. They were receiving a deliv...,0.0
3,3,Stamoolis Brothers is one of the Strip Distric...,0.0
4,1,I want to give a 2 stars because the service s...,0.0
...,...,...,...
9995,1,Four Hours of my life I'd like to have back!\n...,0.0
9996,2,Small stadium and would have great views all a...,0.0
9997,3,I think Porktropolis has the BBQ thing nailed....,0.0
9998,5,Best CU ever!!! Great customer service!!! Know...,1.0


In [6]:
## Getting the dictionary with top 500 words
vectorizer=CountVectorizer(analyzer='word', max_features=1000, tokenizer=word_tokenize, stop_words=stopwords.words('english'))

In [7]:
yelp_X=vectorizer.fit_transform(yelp_df["Review"]).toarray()
print(len(yelp_X[1]))
yelp_y=np.ndarray((10000,), buffer=np.array(yelp_df['pos/neg']))

C:\Users\hatef\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


1000


In [8]:
##Splitting the data
num_training = int(len(yelp_df)*0.75)
print(num_training)
num_testing = int(len(yelp_df)*0.25)

7500


In [9]:
##Splitting the data
yelp_train_X = yelp_X[0:num_training]
yelp_train_y = yelp_y[0:num_training]
yelp_test_X = yelp_X[num_training:]
yelp_test_y = yelp_y[num_training:]

In [10]:
### SVM with different C values
def train_SVM(X, y, c, kernel='linear'):
    print("C=", c)
    clf=SVC(c, kernel='linear')
    clf.fit(X, y)
    return clf

In [11]:
yelp_complete_clf=train_SVM(yelp_X, yelp_y, 1)
yelp_predicted_y=yelp_complete_clf.predict(yelp_X)
print(classification_report(yelp_y, yelp_predicted_y))

C= 1
              precision    recall  f1-score   support

         0.0       0.90      0.91      0.90      6005
         1.0       0.86      0.84      0.85      3995

    accuracy                           0.88     10000
   macro avg       0.88      0.87      0.88     10000
weighted avg       0.88      0.88      0.88     10000



In [ ]:
c=0.00
i=5
c_vals_test=[]
errors_test=[]
c_vals_train=[]
errors_train=[]

## Experimented with different ranges of C values and step sizes and decided to restrict the domain to
## [0.0025, 1] because including other smaller and larger values would take forever  
while(abs(c-1)> 0.005):
    c=0.0005*i
    c_vals_test.append(c)
    c_vals_train.append(c)
    yelp_clf=train_SVM(yelp_train_X, yelp_train_y, c)
    yelp_predicted_test_y=yelp_clf.predict(yelp_test_X)
    yelp_predicted_train_y=yelp_clf.predict(yelp_train_X)
    ##print("*******************************************")
    ##print("train")
    ##print(classification_report(yelp_train_y, yelp_predicted_train_y))
    ##print("*******************************************")
    ##print("test")
    ##print(classification_report(yelp_test_y, yelp_predicted_test_y))
    ##precision, recall, _, _ = precision_recall_fscore_support(yelp_test_y, yelp_predicted_test_y)
    acc=accuracy_score(yelp_train_y, yelp_predicted_train_y)
    accuracy = accuracy_score(yelp_test_y, yelp_predicted_test_y)
    print("training error:", 1 - acc)
    print("Test error: ", 1 - accuracy)
    print("*******************************************")
    errors_train.append(1-acc)
    errors_test.append(1-accuracy)
    i=i+5
e=1
while(abs(e-5) > 0.005):
    e=e+1
    c_vals_test.append(e)
    c_vals_train.append(e)
    yelp_clf=train_SVM(yelp_train_X, yelp_train_y, e)
    yelp_predicted_test_y=yelp_clf.predict(yelp_test_X)
    yelp_predicted_train_y=yelp_clf.predict(yelp_train_X)
    acc=accuracy_score(yelp_train_y, yelp_predicted_train_y)
    accuracy = accuracy_score(yelp_test_y, yelp_predicted_test_y)
    print("training error:", 1 - acc)
    print("Test error: ", 1 - accuracy)
    print("*******************************************")
    errors_train.append(1-acc)
    errors_test.append(1-accuracy)

C= 0.0025
training error: 0.1857333333333333
Test error:  0.19879999999999998
*******************************************
C= 0.005
training error: 0.15973333333333328
Test error:  0.18200000000000005
*******************************************
C= 0.0075
training error: 0.15293333333333337
Test error:  0.18000000000000005
*******************************************
C= 0.01


In [ ]:
x1=np.array(c_vals_test)
y1=np.array(errors_test)
x2=np.array(c_vals_train)
y2=np.array(errors_train)
min_error_test=min(errors_test)
min_error_train=min(errors_train)
print("min error for test =", min_error_test, "at C =",c_vals_test[errors_test.index(min_error_test)] )
print("min error for train =", min_error_train, "at C =",c_vals_train[errors_train.index(min_error_train)] )

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,6))
ax1.scatter(x1,y1)
ax1.set(xlabel='C values')
ax1.set(ylabel='error')
ax1.set_title('Test')

ax2.scatter(x2,y2)
ax2.set(xlabel='C values')
ax2.set(ylabel='error')
ax2.set_title('Training')

plt.show()

## b)

Data is not linearly seperable because the training error is never 0 

## c)

In [ ]:
yelp_sample_X = yelp_X[0:30]
yelp_sample_Y = yelp_y[0:30]
y_clf=train_SVM(yelp_sample_X, yelp_sample_Y, 0.01)
yelp_predicted_sample_y=y_clf.predict(yelp_sample_X)
print(yelp_predicted_sample_y)
print(yelp_sample_Y)
print(yelp_df[0:30])
print("#################################################################################################")
print("A positive sentence that my predictor got right: Cute, quaint coffeehouse with delicious sandwi...")
print("A negative sentence that my predictor got right: Thanks Yelp. I was looking for the words to de...")
print("A positive sentence that my predictor got wrong: I take my kids here at least two times a week")